In [1]:
using Pkg
Pkg.add("Revise")
Pkg.add("XSim")
#Pkg.build("StatsFuns")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/reworkhow/XSim.jl.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [38]:
using DataFrames, Revise
using XSim, DelimitedFiles, Statistics, LinearAlgebra

┌ Info: Precompiling XSim [3d41126b-a46a-5bdb-b7a1-7ea6cc35a8ef]
└ @ Base loading.jl:1186


## Single Trait 

#### Simplest build_genome function: 
only #chromosomes, chromosome length, #markers per chromosome, #QTL per chromosome, #traits, G and mutation rate are parameters, with default values for #traits, G and mutation rate (1, [], 0): markers are equally spaced, all chromosomea are the generated to be equal, gene frequency 0.5 for all loci, #QTL randomly sampled among the markers, QTL effects ~N(0,1)

#### Genotypic values not transformed

In [39]:
numChr=3
chrLength = 1.
numLoci=10
QTLPerChr = 4

4

In [47]:
XSim.build_genome(numChr,chrLength,numLoci,QTLPerChr)

G0 = Any[]


In [48]:
XSim.GLOBAL.G.qtl_effects'XSim.GLOBAL.G.qtl_effects

1×1 Array{Float64,2}:
 13.62274110483927

#### Genotypic values transformed to have variance G

In [49]:
G0 = Array{Float64,2}(undef,1,1)
G0[1,1] = 1
nTraits = 1

1

In [50]:
XSim.build_genome(numChr,chrLength,numLoci,QTLPerChr,nTraits,G0)

G0 = [1.0]
G after transformation = [1.0]


In [51]:
XSim.GLOBAL.G.qtl_effects'XSim.GLOBAL.G.qtl_effects

1×1 Array{Float64,2}:
 1.9999999999999996

#### Intermediate build_genome function: 
still only one chromosome specifiecd and all #chromosomes are generated to be equal, but now gene frequency and map position are vectors with actual values, qtl_index is a vector with the QTL Ids and qtl_effects is a matrix with size #numQTL x #traits.

genotypic values not transformed

In [52]:
geneFreq=fill(0.5,10)

10-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

In [53]:
mPos = sort(rand(10))

10-element Array{Float64,1}:
 0.08870762202455573
 0.19413581353498666
 0.3262914858185366 
 0.41510265827748394
 0.5893216128536118 
 0.6170360621395898 
 0.6184579089148907 
 0.7302950305404872 
 0.7916465068466334 
 0.9282354587998529 

In [54]:
QTLIdx = [1, 3, 5, 6]

4-element Array{Int64,1}:
 1
 3
 5
 6

In [55]:
QTLEffects = randn(4,1)

4×1 Array{Float64,2}:
 -0.03303977103073874
 -0.6390422532270633 
 -0.6608493796621223 
 -0.133537037735719  

In [61]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,0.,QTLIdx,QTLEffects)

G0 = Any[]


In [62]:
XSim.GLOBAL.G.qtl_effects'XSim.GLOBAL.G.qtl_effects

1×1 Array{Float64,2}:
 2.5920620127789853

#### Genotypic values transformed to have variance G

In [58]:
G0 = Array{Float64,2}(undef,1,1)
G0[1,1] = 1

1

In [59]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,0.,QTLIdx,QTLEffects,G0)

G0 = [1.0]
G after transformation = [1.0]


#### Core method for build_genome function: 
#chromosome specified, chromosome_length and nLoci are vectors with the the length of all chromosomes, gene_frequency and map_position are matrices with size #loci x #chromosomes, qtl_index and qtl_effect are vectors of matrices of size #chromosomes x (#QTL x #traits), ntraits, mutation_rate and genotypeErrorRate are scalars and G0 is the genetic covariance matrix the simulated genotypic values will be transformed to. All scalars and the genetic covariance matirx have defaults values (1, 0 and []).

genotypic values not transformed

In [63]:
numChr=3
chrLength = [1.0, 1.1, 0.9]
numLoci=[10 ,15 ,20]
geneFreq   = Array{Array{Float64,1},1}(undef,0)
qtlIndex  = Array{Array{Int64,1},1}(undef,0)
qtlEffects = Array{Array{Float64,2},1}(undef,0)
numQTLOnChr =[2, 3, 1]
numQTL=6
qtlIndex = [[1; 4],[3;7;9],[5]]
geneFreq = [[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5],[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5],[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5]]
mPos = [[.05;.1;.15;.2;.25;.3;.35;.40;.45;.5],[.05;.1;.15;.2;.25;.3;.35;.40;.45;.5;.55;.65;.75;.85;.95],[.05;.1;.15;.2;.25;.3;.35;.4;.45;.5;.54;.58;.65;.75;.758;.759;.8;.85;.851;.859]]
push!(qtlEffects,randn(2,1))
push!(qtlEffects,randn(3,1))
push!(qtlEffects,randn(1,1))


3-element Array{Array{Float64,2},1}:
 [-0.661979; -0.407985]         
 [0.305389; 0.281732; 0.0563519]
 [1.47998]                      

#### Genotypic values not transformed

In [64]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,qtlIndex,qtlEffects)

G0 = Any[]


#### genotypic values transformed to have variance G

In [65]:
G0 = Array{Float64,2}(undef,1,1)
G0[1,1] = 1
nTraits = 1

1

In [66]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,qtlIndex,qtlEffects,nTraits,G0)

G0 = [1.0]
G after transformation = [1.0]


## Two Traits 

#### Simplest build_genome function: 
only #chromosomes, chromosome length, #markers per chromosome, #QTL per chromosome, #traits, G and mutation rate are parameters, with default values for #traits, G and mutation rate (1, [], 0): markers are equally spaced, all chromosomea are the generated to be equal, gene frequency 0.5 for all loci, #QTL randomly dampled among the markers, QTL effects ~N(0,1)

#### genotypic values not transformed

In [67]:
numChr=3
chrLength = 1.
numLoci=10
QTLPerChr = 4
nTraits = 2

2

In [68]:
XSim.build_genome(numChr,chrLength,numLoci,QTLPerChr,nTraits)

G0 = Any[]


#### Genotypic values transformed to have variance G

In [69]:
G0 = [1. 0.5;0.5 2.]

2×2 Array{Float64,2}:
 1.0  0.5
 0.5  2.0

In [70]:
XSim.build_genome(numChr,chrLength,numLoci,QTLPerChr,nTraits,G0)

G0 = [1.0 0.5; 0.5 2.0]
G after transformation = [1.0 0.5; 0.5 2.0]


#### Intermediate build_genome function: 
still only one chromosome specifiecd and all #chromosomes are generated to be equal, but now gene frequency and map position are vectors with actual values, qtl_index is a vector with the QTL Ids and qtl_effects is 2 matrix with size #numQTL x #traits.

#### Genotypic values not transformed

In [71]:
geneFreq=fill(0.5,10)

10-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

In [72]:
mPos = sort(rand(10))

10-element Array{Float64,1}:
 0.030379201594243455
 0.512410942860003   
 0.5567771049861794  
 0.5858531883745721  
 0.6188040481647918  
 0.6642243650188704  
 0.7703515898435962  
 0.8015137136213735  
 0.8264556267549998  
 0.9507417771765398  

In [73]:
QTLIdx = [1, 3, 5, 6]

4-element Array{Int64,1}:
 1
 3
 5
 6

In [74]:
QTLEffects = randn(4,2)

4×2 Array{Float64,2}:
  2.60326   -1.35124   
 -0.406681  -0.816973  
 -1.27219    4.24827e-5
 -0.783233   0.768338  

In [75]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,0.,QTLIdx,QTLEffects)

G0 = Any[]


#### Genotypic values transformed to have variance G

In [76]:
G0 = [1. 0.5;0.5 2.]

2×2 Array{Float64,2}:
 1.0  0.5
 0.5  2.0

In [77]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,0.,QTLIdx,QTLEffects,G0)

G0 = [1.0 0.5; 0.5 2.0]
G after transformation = [1.0 0.5; 0.5 2.0]


#### Core method for build_genome function: 
#chromosome specified, chromosome_length and nLoci are vectors with the the length of all chromosomes, gene_frequency and map_position are matrices with size #loci x #chromosomes, qtl_index and qtl_effect are vectors of matrices of size #chromosomes x (#QTL x #traits), ntraits, mutation_rate and genotypeErrorRate are scalars and G0 is the genetic covariance matrix the simulated genotypic values will be transformed to. All scalars and the genetic covariance matirx have defaults values (1, 0 and []).

#### Genotypic values not transformed

In [79]:
numChr = 3
chrLength = [1.0, 1.1, 0.9]
numLoci = [10 ,15 ,20]
nTraits = 2
geneFreq   = Array{Array{Float64,1},1}(undef,0)
qtlIndex  = Array{Array{Int64,1},1}(undef,0)
qtlEffects = Array{Array{Float64,2},1}(undef,0)
numQTLOnChr =[2, 3, 1]
numQTL=6
qtlIndex = [[1; 4],[3;7;9],[5]]
geneFreq = [[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5],[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5],[.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5;.5]]
mPos = [[.05;.1;.15;.2;.25;.3;.35;.40;.45;.5],[.05;.1;.15;.2;.25;.3;.35;.40;.45;.5;.55;.65;.75;.85;.95],[.05;.1;.15;.2;.25;.3;.35;.4;.45;.5;.54;.58;.65;.75;.758;.759;.8;.85;.851;.859]]
push!(qtlEffects,randn(2,2))
push!(qtlEffects,randn(3,2))
push!(qtlEffects,randn(1,2))


3-element Array{Array{Float64,2},1}:
 [0.739977 -0.272321; -0.30898 -0.864631]                 
 [0.2073 2.19452; 0.189282 0.157835; -0.0813076 -0.935582]
 [-0.216634 -1.53293]                                     

In [80]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,qtlIndex,qtlEffects,nTraits)

G0 = Any[]


#### Genotypic values transformed to have variance G

In [81]:
G0 = [1. 0.5;0.5 2.]

2×2 Array{Float64,2}:
 1.0  0.5
 0.5  2.0

In [82]:
XSim.build_genome(numChr,chrLength,numLoci,geneFreq,mPos,qtlIndex,qtlEffects,nTraits,G0)

G0 = [1.0 0.5; 0.5 2.0]
G after transformation = [1.0 0.5; 0.5 2.0]
